In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [100]:
dataset = pd.read_csv('./dataset/Churn_Modelling.csv')

In [101]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# preprocessing 

In [102]:
# drop irrelevant columns
dataset = dataset.drop(['RowNumber','CustomerId', 'Surname'], axis=1) # 1 means column wise

In [103]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### convert catagorical variables into numerical


In [104]:
# gender column into 0,1:
label_encoder_gender = LabelEncoder()
dataset['Gender'] = label_encoder_gender.fit_transform(dataset['Gender']) # if we store into another var new column will be created.

In [105]:
dataset.head() # gender is converted into 0(female),1(male)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [106]:
# Gerography column has 3 catogories (france, germany, spain)
# converting into one hot encoding

from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)
geo_encoder = onehot_encoder.fit_transform(dataset[['Geography']])

In [107]:
print(dataset['Geography'].shape) # 1d vector
print(dataset[['Geography']].shape) # 2d matrix

(10000,)
(10000, 1)


In [108]:
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [109]:
geo_catagories = onehot_encoder.get_feature_names_out(['Geography']) # it has 3 samples that are converted into features
geo_catagories


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [110]:
# now we use these columns as features and insert geo_encoder values init.

geo_encoded_df = pd.DataFrame(geo_encoder, columns= geo_catagories)

In [111]:
# geo_encoder values are inserted into geo_catagories features
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [112]:
# we have to insert the above features into the dataset
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [113]:
# drop geography column from dataset, coz we will replace it with geo_encoded_df
drop_geo = dataset.drop(['Geography'], axis=1) # 1 means column


In [114]:
drop_geo.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0


In [115]:
dataset = pd.concat([drop_geo, geo_encoded_df], axis=1)

In [116]:
# features are added into the dataset
dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


# saving encoder and scaler into file:

In [117]:

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)

# split dataset into independent and dependent variables
- x = (all columns) => independent
- y = (['Exited'] column) => dependent 

In [22]:
x = dataset.drop(['Exited'], axis=1)
y = dataset[['Exited']] # keep it in 2d matrix like row, column wise, not in scaler form that's why [['Exited']]
x.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [23]:
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


# spliting this into test and train

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) # keeping test data 20%

# feature scaling

In [25]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [26]:
print(x_train)

[[ 0.35649971  0.91324755 -0.6557859  ...  1.00150113 -0.57946723
  -0.57638802]
 [-0.20389777  0.91324755  0.29493847 ... -0.99850112  1.72572313
  -0.57638802]
 [-0.96147213  0.91324755 -1.41636539 ... -0.99850112 -0.57946723
   1.73494238]
 ...
 [ 0.86500853 -1.09499335 -0.08535128 ...  1.00150113 -0.57946723
  -0.57638802]
 [ 0.15932282  0.91324755  0.3900109  ...  1.00150113 -0.57946723
  -0.57638802]
 [ 0.47065475  0.91324755  1.15059039 ... -0.99850112  1.72572313
  -0.57638802]]


# storing feature scaling into file:


In [27]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [28]:
dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


# building ANN model

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [36]:
print(x_train.shape)
print(x_train[1].shape)
print(x_train[1])

(8000, 12)
(12,)
[-0.20389777  0.91324755  0.29493847 -0.3483691   0.69683765  0.80843615
  0.64920267  0.97481699  1.6612541  -0.99850112  1.72572313 -0.57638802]


### creating model


In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train[1].shape)), # 1st hidden layer
    Dense(32, activation='relu'), # 2nd hidden layer
    Dense(1, activation='sigmoid') # output layer
])

In [44]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

### compile the model

In [66]:
model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

### setup tensorboard

In [54]:
import datetime
datetime.datetime.now()

datetime.datetime(2025, 3, 26, 2, 33, 16, 615212)

In [85]:
file_path = 'logs/fir/' + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path

'logs/fir/2025-03-26_03-00-05'

In [86]:
tensorflow_callback = TensorBoard(log_dir=file_path, histogram_freq=1)

In [87]:
tensorflow_callback

### setup early stopping

In [63]:
early_stoping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 
# wait 10 epochs to check the changes in val_loss
# if it's very low, it stops training process
# and stores the w values, which has less loss

### Train model

In [88]:
history = model.fit(
        x_train, y_train, validation_data= (x_test, y_test), epochs=100,
        callbacks = [tensorflow_callback, early_stoping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6851 - loss: 0.5677 - val_accuracy: 0.8285 - val_loss: 0.3961
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8314 - loss: 0.4034 - val_accuracy: 0.8565 - val_loss: 0.3590
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8491 - loss: 0.3644 - val_accuracy: 0.8580 - val_loss: 0.3485
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8550 - loss: 0.3482 - val_accuracy: 0.8585 - val_loss: 0.3452
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8636 - loss: 0.3354 - val_accuracy: 0.8580 - val_loss: 0.3455
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8649 - loss: 0.3334 - val_accuracy: 0.8620 - val_loss: 0.3469
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8548 - loss: 0.3414 - val_accuracy: 0.8625 - val_loss: 0.3407
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8582 - loss: 0.3446 - val_accu

In [89]:
# saving model file
model.save("model.h5")

# Load tensorboard extension

In [90]:
%load_ext tensorboard

In [95]:
%tensorboard --logdir logs/fir/2025-03-26_03-00-05

Reusing TensorBoard on port 6013 (pid 14308), started 0:00:28 ago. (Use '!kill 14308' to kill it.)

In [97]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,837 (34.52 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,892 (23.02 KB)